# 1. Import Dependencies

In [65]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- thumbsup
- thumbsdown
- thankyou
- livelong

The information about each hand gesture is contained into 42 features.

In [66]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_1.csv")

In [67]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [68]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
35,419,184,391,187,372,177,367,166,366,156,...,49,444,123,450,99,454,83,0,0,3
37,652,465,603,461,561,430,534,399,511,372,...,206,704,365,727,324,742,298,0,0,3
22,553,451,603,433,647,397,691,373,737,365,...,205,489,377,477,321,470,285,0,0,2
2,357,442,358,388,339,353,310,327,293,299,...,420,288,466,273,452,281,443,0,0,0
21,561,409,610,411,653,388,688,372,728,356,...,195,523,353,511,305,503,274,0,0,2


In [69]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
35,419,184,391,187,372,177,367,166,366,156,...,407,49,444,123,450,99,454,83,0,0
37,652,465,603,461,561,430,534,399,511,372,...,678,206,704,365,727,324,742,298,0,0
22,553,451,603,433,647,397,691,373,737,365,...,484,205,489,377,477,321,470,285,0,0
2,357,442,358,388,339,353,310,327,293,299,...,281,420,288,466,273,452,281,443,0,0
21,561,409,610,411,653,388,688,372,728,356,...,503,195,523,353,511,305,503,274,0,0


In [70]:
train.shape  # we have 120 entires with 4 features

(31, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [79]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [80]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [81]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [82]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [83]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 325.46793, step = 0
INFO:tensorflow:global_step/sec: 89.2806
INFO:tensorflow:loss = 9.010386, step = 100 (1.123 sec)
INFO:tensorflow:global_step/sec: 125.463
INFO:tensorflow:loss = 4.7140303, step = 200 (0.795 sec)
INFO:tensorflow:global_step/sec: 149.468
INFO:tensorflow:loss = 2.6572094, step = 300 (0.670 sec)
INFO:tensorflow:global_step/sec: 201.196
INFO:tensorflow:loss = 1.7965034, step = 400 (0.497 sec)
INFO:tensorflow:global_step/sec: 188.313
INFO:tensorflow:loss = 1.266749

## Export Model

In [85]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1635164294\saved_model.pbtxt


## Load Model

In [117]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x42C87A90>

In [143]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert form list to tensor
examples = tf.constant(examples)

print(test)

      0    1    2    3    4    5    6    7    8    9  ...   32   33   34   35  \
4   610  360  612  326  601  294  592  269  590  243  ...  551  426  619  415   
12  757  227  735  265  708  297  683  320  671  339  ...  689  224  677  194   
14  758  248  744  288  708  320  680  343  663  366  ...  614  225  678  195   
15  768  264  735  313  705  343  673  363  654  381  ...  666  243  663  201   
16  416  273  413  330  372  376  342  411  323  441  ...  268  259  306  211   
25  698  310  751  296  787  259  817  230  847  215  ...  653   66  652  228   
26  625  349  656  332  681  303  702  278  722  262  ...  573  191  583  295   
27  622  358  655  334  680  304  703  279  724  263  ...  570  193  580  297   

     36   37   38   39  40  41  
4   579  437  562  440   0   0  
12  652  198  673  204   0   0  
14  640  192  633  192   0   0  
15  623  203  647  215   0   0  
16  256  214  262  220   0   0  
25  636  175  625  143   0   0  
26  568  260  559  239   0   0  
27  56

In [145]:
predictions = predict_fn(examples=examples)
print(predictions)

{'all_class_ids': <tf.Tensor: shape=(8, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])>, 'class_ids': <tf.Tensor: shape=(8, 1), dtype=int64, numpy=
array([[0],
       [1],
       [1],
       [1],
       [1],
       [3],
       [2],
       [2]], dtype=int64)>, 'classes': <tf.Tensor: shape=(8, 1), dtype=string, numpy=
array([[b'0'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'3'],
       [b'2'],
       [b'2']], dtype=object)>, 'all_classes': <tf.Tensor: shape=(8, 4), dtype=string, numpy=
array([[b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3'],
       [b'0', b'1', b'2', b'3']], dtype=object)>, 'logits': <tf.Tensor: shape=(8, 4), dtype=float32, numpy

In [204]:
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.9678871, shape=(), dtype=float32)
	Prediction is thumbsup 96.79%
	Expected: thumbsup
tf.Tensor(0.99632627, shape=(), dtype=float32)
	Prediction is thumbsdown 99.63%
	Expected: thumbsdown
tf.Tensor(0.99999976, shape=(), dtype=float32)
	Prediction is thumbsdown 100.00%
	Expected: thumbsdown
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is thumbsdown 100.00%
	Expected: thumbsdown
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is thumbsdown 100.00%
	Expected: thumbsdown
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is livelong 100.00%
	Expected: thankyou
tf.Tensor(0.99999964, shape=(), dtype=float32)
	Prediction is thankyou 100.00%
	Expected: thankyou
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is thankyou 100.00%
	Expected: thankyou


## Evaluation

In [55]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-19T20:37:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Umberto\AppData\Local\Temp\tmpe8e28tod\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.23201s
INFO:tensorflow:Finished evaluation at 2021-10-19-20:37:57
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.375, average_loss = 6.9146185, global_step = 5000, loss = 6.9146185
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\Umberto\AppData\Local\Temp\tmpe8e28tod\model.ckpt-5000

Test set accuracy: 0.375



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [74]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Umberto\AppData\Local\Temp\tmpe8e28tod\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "thumbsup" (100.0%)
Expected is thumbsup
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Umberto\AppData\Local\Temp\tmpe8e28tod\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "thumbsup" (51.5%)
Expected is thumbsdown
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Umberto\AppData\Local\Temp\tmpe8e28tod\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running loc